[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shimizu-biz/nlp_workshop/blob/master/step2.ipynb)

# step 2

不要な警告を非表示にする

In [0]:
import warnings
warnings.filterwarnings('ignore')

gensimのインストールを行いましょう
gensimは自然言語処理のためにしばしば使われる（ベクトル空間モデル、トピックモデル）ライブラリです

In [0]:
%%bash
pip3 install gensim

gensimのインポート

In [0]:
import gensim
from gensim.models import word2vec

本来は学習のプロセスも体験していただきたいのですが、学習にはかなり時間がかかるので、有志の方の学習済みもモデルを使わせてもらいます

In [0]:
%%bash
curl -L -o pretrained_model.bin.zip  "http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip"
unzip pretrained_model.bin.zip

モデルのロード

In [0]:
model = word2vec.Word2Vec.load('word2vec.gensim.model')

語彙数の確認しましょう

In [0]:
len(model.wv.vocab)

語彙（一部）を表示しましょう

In [0]:
model.wv.vocab

東京にどのようなベクトルが割り当てられているか確認しましょう

In [0]:
model.wv['東京']

コサイン類似度に基づいて、語彙の中で最も「東京」に近い単語を表示しましょう

In [0]:
model.wv.most_similar(positive=["東京"], topn=1)

Question:「大きい」に最も近い単語は何でしょうか？

1. 大きな
2. 大
3. 大きさ
3. 小さい

In [0]:
model.wv.most_similar(positive=["大きい"], topn=1)

他の候補はどうだったのでしょうか？

In [0]:
print (model.wv.similarity('大きい','大きな'))
print (model.wv.similarity('大きい','大'))
print (model.wv.similarity('大きい','大きな'))
print (model.wv.similarity('大きい','小さい'))

ベクトルなので足し算・引き算ができます。
日本にとっての東京は、アメリカにとっての何でしょうか？

In [0]:
vector = model.wv['東京'] - model.wv['日本'] + model.wv['アメリカ']
model.wv.similar_by_vector(vector, topn=1, restrict_vocab=None)

文のベクトルを表現する簡単な方法の一つとして、単語のベクトルの和を考えることができます。
これにより、文同士のコサイン類似度を計算することもできます。

In [0]:
import MeCab
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def mas_ls(text):#mecabを使って分かち書き
    m = MeCab.Tagger("-Owakati")
    return m.parse(text)[:-1]
def word_vector_sum(ls):#単語ベクトルの和を計算
    vec = np.zeros(model.wv.vector_size)
    for word in ls:
        try:
            vec += model.wv[word]
        except:
            pass
    return vec
def calc_sim(s1,s2):
    v1 = word_vector_sum(mas_ls(s1))
    v2 = word_vector_sum(mas_ls(s2))
    return cosine_similarity(v1.reshape(1,-1),v2.reshape(1,-1))#ベクトルのコサイン類似度を計算

文のペアが二つあります。どちらのペアが似ている組み合わせだと思いますか。

In [0]:
s1 = "今日は晴れだ"
s2 = "本日は快晴だ"
print (calc_sim(s1,s2))

s1 = "今日は晴れだ"
s2 = "好きな２文を入れてみましょう"
print (calc_sim(s1,s2))

この方法も、語順の情報が失われてしまいます

In [0]:
s1 = "りんごは好きだが、ぶどうは嫌いだ"
s2 = "ぶどうは好きだが、りんごは嫌いだ"
print (calc_sim(s1,s2))